이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7 

Defaulting to user installation because normal site-packages is not writeable


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("local[3]") \
        .enableHiveSupport() \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/25 17:20:05 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/25 17:20:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/25 17:20:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/25 17:20:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-01-29 04:24:54--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.218.228.225, 52.92.194.74, 52.218.220.193, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.218.228.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv.1’

appl_stock.csv.1    100%[===================>] 139.78K   765KB/s    in 0.2s    

2023-01-29 04:24:54 (765 KB/s) - ‘appl_stock.csv.1’ saved [143130/143130]



In [3]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [4]:
df.show(10)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [5]:
df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

In [6]:
spark.sql("DROP TABLE IF EXISTS appl_stock")

24/06/25 17:22:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/25 17:22:40 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/06/25 17:22:49 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/06/25 17:22:49 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore sangwon@127.0.1.1
24/06/25 17:22:49 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


DataFrame[]

In [7]:
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

24/06/25 17:23:00 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/06/25 17:23:00 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/06/25 17:23:00 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/25 17:23:00 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/06/25 17:23:04 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [8]:
!ls -tl 

total 472
drwxr-xr-x 3 sangwon sangwon   4096 Jun 25 17:22 spark-warehouse
drwxr-xr-x 5 sangwon sangwon   4096 Jun 25 17:22 metastore_db
-rw-r--r-- 1 sangwon sangwon    814 Jun 25 17:22 derby.log
-rw-r--r-- 1 sangwon sangwon   1716 Jun 25 17:16 bucketDemo.py
-rw-r--r-- 1 sangwon sangwon  18161 Jun 25 16:58 PySpark_FileSystem_Partitioning.ipynb
-rw-r--r-- 1 sangwon sangwon  22795 Jun 25 16:41 PySpark_Schema_Evolution.ipynb
-rw-r--r-- 1 sangwon sangwon  15644 Jun 25 16:38 PySpark_Write_실습.ipynb
drwxr-xr-x 5 sangwon sangwon   4096 Jun 25 16:36 dataOutput
drwxr-xr-x 2 sangwon sangwon   4096 Jun 20 01:17 wordcount
drwxr-xr-x 4 sangwon sangwon   4096 Jun 20 01:17 join_broadcast_join
-rw-r--r-- 1 sangwon sangwon  31964 Jun 20 01:17 PySpark_ML_실습1.ipynb
-rw-r--r-- 1 sangwon sangwon  29648 Jun 20 01:17 PySpark_ML_실습2.ipynb
-rw-r--r-- 1 sangwon sangwon  71580 Jan 26  2023 schema3.parquet
-rw-r--r-- 1 sangwon sangwon  49915 Jan 26  2023 schema1.parquet
-rw-r--r-- 1 sangwon sangwon  57

In [9]:
!ls -tl spark-warehouse/

total 4
drwxr-xr-x 9 sangwon sangwon 4096 Jun 25 17:23 appl_stock


In [10]:
!ls -tl spark-warehouse/appl_stock/

total 28
-rw-r--r--  1 sangwon sangwon    0 Jun 25 17:23  _SUCCESS
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:23 'year=2016'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2015'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2014'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2013'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2012'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2011'
drwxr-xr-x 14 sangwon sangwon 4096 Jun 25 17:22 'year=2010'


In [11]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 48
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=12'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=11'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=10'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=9'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=8'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=7'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=6'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=5'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=4'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=3'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=2'
drwxr-xr-x 2 sangwon sangwon 4096 Jun 25 17:22 'month=1'


In [12]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 sangwon sangwon 3015 Jun 25 17:22 part-00000-b2d4ba6e-8f82-4662-a4af-e9a1095e0b05.c000.snappy.parquet


### How to Read from Partitioned table

In [13]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [14]:
df.show(10)

+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|      Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08|        110.860001|            112.43|110.599998|112.120003|27068300|111.63599599999999|2016

In [15]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)

+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|      Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08|        110.860001|            112.43|110.599998|112.120003|27068300|111.63599599999999|2016